# Test Envoiriment for the  functions in the module

[How To Use Conda Environment In a Jupyter Notebook
](https://saturncloud.io/blog/how-to-use-conda-environment-in-a-jupyter-notebook/#:~:text=Using%20a%20Conda%20Environment%20in%20Jupyter%20Notebook&text=This%20package%20allows%20you%20to,be%20used%20in%20Jupyter%20Notebook.&text=This%20will%20create%20a%20new%20kernel%20for%20the%20“myenv”%20environment,be%20used%20in%20Jupyter%20Notebook.)

## Train

In [ ]:
#!python main.py --model --datasetpath --state "train" --pretrained_path --output_path

## Test

In [ ]:
from loss import *
from Metrics import *
from torch.nn.functional import mse_loss
from kornia.losses import ssim_loss
from numpy import mean 
from time import time

class ssim_1(nn.Module):
    def __init__(self, window_size):
        super(ssim_1,self).__init__()
        self.window_size = window_size
    def forward(self, input, target):
        return ssim_loss(input, target, self.window_size)

class mse_1(nn.Module):
    def __init__(self):
        super(mse_1,self).__init__()

    def forward(self, input, target):
        return mse_loss(input, target)

gt_image = torch.randn(1, 3, 64, 64, requires_grad=True)
t_image = torch.randn(1, 3, 64, 64, requires_grad=True)
mean_val = mean(t_image.detach().numpy())

# Example usage of the Dark Channel Prior loss function
#find patch size
patch_size = 5
print(mean)
losses = {
    "DarkChannelPriorLoss": DarkChannelPriorLoss(patch_size),
    "L_exp": L_exp(patch_size,mean_val),
    "L_color": L_color(),
    "mse_loss":mse_1(),
    "light_loss": light_loss2(),
    "color_loss": color_loss2(),
    "L1Loss": L1Loss(),
    "SSIM loss":ssim_1(window_size=11),
    "lch_loss": lch_Loss(),
    "lab_loss": lab_Loss(),
    "loss_fn_alex" : LPIPS(net='alex'), # best forward scores
    "loss_fn_vgg" : LPIPS(net='vgg'), # closer to "traditional" perceptual loss, when used for optimization
    "loss_squeeze" : LPIPS(net='squeeze') # lightest, faster, narrower, lower quality
}

for name, loss in losses.items():
    start_time = time()
    if name== "loss_fn_vgg" or name == "loss_squeeze" or name == "loss_fn_alex" or name == "L1Loss" or name == "light_loss" or name == "color_loss" or name == "SSIM loss" or name == "color_loss" or name == "mse_loss" or name == "lab_loss" or name == "lch_loss":
        loss = loss(gt_image, t_image)
    else:
        loss = loss(t_image)
    end_time = time()
    print(name, ":", loss.item(), "// Time:", end_time-start_time)



In [3]:
!pip install colorspacious

### TESTAR E VALIDAR O RESULTADO DO LAB LOSS E LCH LOSS

In [7]:
import torch
import colorspacious as cs

def rgb_to_lab(image):
    # Assumindo que a imagem está em formato (N, C, H, W)
    image = image.permute(0, 2, 3, 1).detach().numpy()  # Convertendo para (N, H, W, C) e numpy
    image_lab = cs.cspace_convert(image, "sRGB1", "CIELab")
    return torch.tensor(image_lab).permute(0, 3, 1, 2).to(device)  # Convertendo de volta para tensor PyTorch

def lab_to_lch(lab_image):
    lab_image = lab_image.permute(0, 2, 3, 1).detach().numpy()  # Convertendo para (N, H, W, C) e numpy
    lch_image = cs.cspace_convert(lab_image, "CIELab", "CIELCh")
    return torch.tensor(lch_image).permute(0, 3, 1, 2).to(device)  # Convertendo de volta para tensor PyTorch

def calculate_lab_loss(gt_image, t_image):
    gt_lab = rgb_to_lab(gt_image)
    t_lab = rgb_to_lab(t_image)
    lab_loss = torch.sqrt(torch.sum((gt_lab - t_lab) ** 2, dim=1)).mean()
    return lab_loss

def calculate_lch_loss(gt_image, t_image):
    gt_lab = rgb_to_lab(gt_image)
    t_lab = rgb_to_lab(t_image)
    gt_lch = lab_to_lch(gt_lab)
    t_lch = lab_to_lch(t_lab)
    lch_loss = torch.sqrt(torch.sum((gt_lch - t_lch) ** 2, dim=1)).mean()
    return lch_loss

# Exemplo de uso
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Exemplo de uso
gt_image = torch.randn(1, 3, 64, 64, requires_grad=True)
t_image = torch.randn(1, 3, 64, 64, requires_grad=True)

lab_loss = calculate_lab_loss(gt_image, t_image)
lch_loss = calculate_lch_loss(gt_image, t_image).cpu()

print(f'LAB Loss: {lab_loss.item()}')
print(f'LCH Loss: {lch_loss.item()}')

###

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

### /TESTE DE /METRICAS

In [10]:
from skimage.measure import compare_psnr, compare_ssim
import skimage

In [8]:
!cd Metrics && python metrics.py "data/UDWdata/SUIM/train_val/images"

Traceback (most recent call last):
  File "/home/gusanagy/Documents/Glown-Diffusion/Metrics/metrics.py", line 15, in <module>
    from skimage.measure import compare_psnr, compare_ssim
ImportError: cannot import name 'compare_psnr' from 'skimage.measure' (/home/gusanagy/anaconda3/envs/CLEDiff/lib/python3.11/site-packages/skimage/measure/__init__.py)


### Testes de datasets

In [1]:
from src.split_data import split_dataset

dataset_size = 1000  # Tamanho do dataset
proportions = (0.7, 0.2, 0.1)  # Proporções para treino, teste e validação
dataset_path = "data/UDWdata/SUIM/train_val/images"  # Caminho para a pasta do dataset

result = split_dataset(dataset_size, proportions, dataset_path)

print(f"Imagens para treino: {result['train_size']}")
print(f"Imagens para teste: {result['test_size']}")
print(f"Imagens para validação: {result['val_size']}")

print("Endereços das imagens para treino:")
for img in result['train_images']:
    print(img)

print("Endereços das imagens para teste:")
for img in result['test_images']:
    print(img)

print("Endereços das imagens para validação:")
for img in result['val_images']:
    print(img)

1525
1000


TypeError: 'NoneType' object is not subscriptable

## Inference